In [1]:
import os

In [2]:
%pwd

'd:\\MLops_CNN_Classifier_END_PROJECT\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\MLops_CNN_Classifier_END_PROJECT'

In [5]:
# 1.entity
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path 
    unzip_dir: Path

In [6]:
# configuration manager
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml,create_directories

In [7]:
# constants
from pathlib import Path 

config_file_path=Path("config/config.yaml")
params_file_path=Path("params.yaml")

In [8]:
# configuration manager 2
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= config_file_path,
        param_filepath=  params_file_path):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion    

        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [9]:
# components
import os 
import zipfile
import gdown 
from CNNClassifier import logger 
from CNNClassifier.utils.common import get_size

In [10]:
# datset download
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    def download_file(self)-> str:
        try:
            dataset_url=self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file{zip_download_dir}")

            file_id=dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e 
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    

In [11]:
# pipeline
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-06 00:25:04,455: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-06 00:25:04,508: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-06 00:25:04,511: INFO: common: created directory at: artifacts]
[2024-02-06 00:25:04,517: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-06 00:25:04,520: INFO: 4106099252: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into fileartifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=612da976-3399-44a4-acae-092db2e075ee
To: d:\MLops_CNN_Classifier_END_PROJECT\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:18<00:00, 2.60MB/s]


[2024-02-06 00:25:26,217: INFO: 4106099252: Download data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
